In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys, os.path, os
path_opencp="drive/Shareddrives/FAIRNESS/Colab/Librerias/PredictCode/"
path_fairness="drive/Shareddrives/FAIRNESS/Colab/Librerias/"
path_exp="drive/Shareddrives/FAIRNESS/Colab/Notebooks_for_topic/EXP0/SCRIPTS/"
sys.path.insert(0, os.path.abspath(path_opencp))
sys.path.insert(0, os.path.abspath(path_fairness))
sys.path.insert(0, os.path.abspath(path_exp))

Mounted at /content/drive


In [ ]:
import pickle as pkl
import os
import datetime
import numpy as np
from datetime import timedelta
import open_cp
import pandas as pd

from google.colab import output

#Cargando los datos
os.chdir("drive/Shareddrives/FAIRNESS/Colab/Notebooks_for_topic/EXP0/")

from models.model_selection import NAIVE_MODEL, KDE_MODEL, SEPP_MODEL

Failed to load 'descartes' package.
ERROR:open_cp.geometry:Failed to import `rtree`.
ERROR:open_cp.network:Failed to import `rtree`.


## 0. Setup variables

In [ ]:
from open_cp.sources.sepp import make_time_unit
from global_vars import x_min,x_max,y_min,y_max,grid_size,days_time_unit

region = open_cp.RectangularRegion(x_min,x_max, y_min,y_max)

# cut_time=24*15
# cut_space=0.5

from global_vars import dir_sims, dir_split, f_final_train

from models.model_selection import NAIVE_MODEL, KDE_MODEL, SEPP_MODEL

In [ ]:

def intensity_m(model,date,region=region,grid_size=grid_size):

    try:
        predict=model.predict(date)
    except:
        predict=model.predict()

    try:
        M=open_cp.predictors.GridPredictionArray.from_continuous_prediction_region(predict, region, grid_size, grid_size)
        intensity_matrix= M.intensity_matrix
    except:
        intensity_matrix = predict.intensity_matrix

    if intensity_matrix.sum()==0:
        return intensity_matrix

    else:
        return intensity_matrix/intensity_matrix.sum()

### datos reales
import open_cp.naive as naive
def get_real_m(timedpoints,p_date,grid_size=grid_size,region=region):
    real = timedpoints[(timedpoints.times_datetime()>=p_date) & ((timedpoints.times_datetime()<p_date+timedelta(days=1)))]
    predictor = naive.CountingGridKernel(grid_size,region=region)
    predictor.data = real
    try:
        gridpred = predictor.predict().renormalise()
        real=gridpred.intensity_matrix.data

        return real
    except:
        return predictor.predict().intensity_matrix


from scipy.optimize import linprog
from math import sqrt
def EMD(m1,m2,cuadricula):
    P=[]
    Q=[]
    coordenadas = []

    #-------------------------------- PARTE 1 --------------------------------
    #------funcion objetivo
    #Como m1,m2 son del mismo tamaño no hay problemas con el bucle
    paso_filas = cuadricula/2
    for i in range(0,m1.shape[0]):
        paso_columna = cuadricula/2
        for j in range(0, m1.shape[1]):
            coordenadas.append([paso_columna,paso_filas])
            P.append([[paso_columna,paso_filas],m1[i][j]])
            Q.append([[paso_columna,paso_filas],m2[i][j]])
            paso_columna += cuadricula
        paso_filas += cuadricula

    #Inicializando y generando nuestra funcion objetivo
    obj = []
    for i in range(0, len(P)):
        for j in range(0, len(Q)):
            obj.append(sqrt( pow(P[i][0][0]-Q[j][0][0],2)+pow(P[i][0][1]-Q[j][0][1],2) ))


    #-------------------------------- PARTE 2 --------------------------------
    #------restricciones inecuaciones
    #Inicializando los constrains
    lhs_ineq = []
    #constrains de fila
    for m in range(0, len(P)):
        aux = np.zeros((len(P), len(Q)))
        aux[m:m+1] = np.ones((1,len(P)))
        lhs_ineq.append(np.asarray(aux).reshape(-1))
    #constrains de columnas
    for m in range(0, len(Q)):
        aux = np.zeros((len(P), len(Q)))
        aux[:,m:m+1] = np.ones((len(Q),1))
        lhs_ineq.append(np.asarray(aux).reshape(-1))

    #Calculando las restricciones de los constrains
    rhs_ineq = []
    #constrains de fila
    for m in range(0, len(P)):
        rhs_ineq.append(P[m][1])
    #constrains de columnas
    for m in range(0, len(Q)):
        rhs_ineq.append(Q[m][1])


    #-------------------------------- PARTE 3 --------------------------------
    #------restricciones ecuaciones
    lhs_eq = [[1 for i in range(0, len(P)*len(Q))]]
    rhs_eq = [min(sum([i[1] for i in P]), sum([i[1] for i in Q]))]


    #-------------------------------- PARTE 4 --------------------------------
    #------limites de las variables f_ij
    bnd = [(0, 1) for i in range(0, len(P)*len(Q))]


    #-------------------------------- PARTE 5 --------------------------------
    # opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq, A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd, method="revised simplex")
    opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq, A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd, method="highs")
    solucion = opt.x.reshape(len(P),len(Q))



    #-------------------------------- CALCULO EMD --------------------------------
    #------Calculando la earth mover distance
    emd = 0
    for i in range(0, len(P)):
        for j in range(0, len(Q)):
            emd += (solucion[i][j])*( sqrt( pow(P[i][0][0]-Q[j][0][0],2)+pow(P[i][0][1]-Q[j][0][1],2) ) )
    emd = emd/solucion.sum().sum()

    return (emd)

In [ ]:
### grid search


### SEPP

## hours        1  12  24  24*5    24*15   24*30   24*90
## cut_space    sqrt(2)     1       sqrt(2)/2   0.5     0.1

params={
    "hours":[24*30*6],
    "cutoff":[grid_size]
}



In [ ]:
from global_vars import dir_models

path_models_sepp=os.path.join(dir_models,"SEPP")

if not os.path.exists(path_models_sepp):
    os.mkdir(path_models_sepp)


table_path=os.path.join(path_models_sepp,"SEPP_EMD.xlsx")

if os.path.exists ( table_path ):

    table= pd.read_excel(table_path)

else:

    table=pd.DataFrame(columns=["Data_id","EMD"])


for id in range(0,30):
    output.clear(wait=True)
    print("-"*30)
    print("data:",id)

    if len(table.loc[table["Data_id"]==id])>0:
        print("Termino entrenamiento")
        print("Termino prediccion y medida EMD")
        continue


    # path train
    train_path=os.path.join(dir_split,"Train_Data_"+str(id)+".pkl")
    # path train
    test_path=os.path.join(dir_split,"Test_Data_"+str(id)+".pkl")


    ## carga train
    train = pkl.load(open(train_path, "rb"))
    train =train[(train.xcoords<=1) & (train.xcoords>=0) & ((train.ycoords<=1)) & (train.ycoords>=0)]

    ## carga test
    test = pkl.load(open(test_path, "rb"))
    test =test[(test.xcoords<=1) & (test.xcoords>=0) & ((test.ycoords<=1)) & (test.ycoords>=0)]


    path_models_sepp_data=os.path.join(dir_models,"SEPP","Data_"+str(id))

    if not os.path.exists(path_models_sepp_data):
        os.mkdir(path_models_sepp_data)


    days_test=int((test.time_range[1]-test.time_range[0]).astype('timedelta64[D]')/np.timedelta64(1, 'D'))

    for times in params["hours"]:
        print("times:",times)
        for spaces in params["cutoff"]:
            print("cutoff:",spaces)



            prms={"hours":times,"cutoff":spaces}

            path_file=os.path.join(path_models_sepp_data,"Data_"+str(id)+".pkl")

            if not os.path.exists(path_file):
                sepp_model_result=SEPP_MODEL(train,20,times,spaces)
                pkl.dump(sepp_model_result, open(path_file, "wb"))

            else:
                sepp_model_result=pkl.load(open(path_file, "rb"))

            print("Termino entrenamiento")
            predictions={}
            reals={}
            EMD_mean=[]
            for i in range(days_test):
                pred_date=f_final_train+timedelta(days=i)
                # predict=np.array([intensity_m(sepp_model_result,pred_date) for i in range(200)]).mean(axis=0)
                predict=intensity_m(sepp_model_result,pred_date)
                predictions[pred_date]=predict
                # real data
                real=get_real_m(test,pred_date)
                reals[pred_date]=real
                EMD_mean.append(EMD(real,predict,1/25))

            pkl.dump(predictions, open(os.path.join(path_models_sepp_data,"predictions.pkl"), "wb"))
            pkl.dump(reals, open(os.path.join(path_models_sepp_data,"reals.pkl"), "wb"))

            print("Termino prediccion y medida EMD")

            EMD_mean=np.array(EMD_mean).mean()

            try:
                table= pd.read_excel(table_path)
            except:
                table=pd.DataFrame(columns=["Data_id","EMD"])

            table.loc[len(table)]=[id,EMD_mean]
            table.to_excel(table_path,index=False)